In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ExponentialSmoothing
import imageio
import os

# Generate synthetic time series data
np.random.seed(42)
time_index = pd.RangeIndex(start=0, stop=100, step=1)  # Use a pandas RangeIndex
values = 2 * np.sin(0.1 * np.arange(100)) + np.random.normal(0, 0.5, size=100)

# Convert the data into a TimeSeries object
series = TimeSeries.from_times_and_values(time_index, values)

# Split data into training and test sets
train, test = series.split_before(0.8)

# Use Exponential Smoothing for forecasting
model = ExponentialSmoothing()
model.fit(train)

# Create folder to store frames
if not os.path.exists("forecast_frames"):
    os.makedirs("forecast_frames")

# Generate enough predictions to sustain 1 minute of animation
forecast_frames = []
total_frames = 300  
forecast_steps = 200  

for i in range(1, total_frames + 1):
    # Incrementally forecast more steps as we progress
    pred = model.predict(i if i <= forecast_steps else forecast_steps)
    combined_series = train.append(pred) 

    plt.figure(figsize=(10, 6))
    train.plot(label="Train", color="green")
    test.plot(label="Test", color="orange")
    combined_series.plot(label="Forecast", color="blue")
    
    # Extract 1D arrays for fill_between
    pred_values = pred.values().flatten()  
    plt.fill_between(
        pred.time_index,
        pred_values - 0.5,
        pred_values + 0.5,
        alpha=0.2,
        color="blue"
    )
    
    plt.legend()
    plt.title("Time Series Forecasting (Continuous)")
    plt.xlabel("Time Index")
    plt.ylabel("Target Value")
    frame_file = f"forecast_frames/frame_{i:03d}.png"
    plt.savefig(frame_file)
    plt.close()
    forecast_frames.append(frame_file)

# Combine frames into a GIF
gif_file = "time_series_forecast_continuous.gif"
with imageio.get_writer(gif_file, mode="I", duration=0.2) as writer:
    for frame_file in forecast_frames:
        writer.append_data(imageio.imread(frame_file))

for frame_file in forecast_frames:
    os.remove(frame_file)

print(f"Continuous Forecast GIF saved as {gif_file}")
